In [17]:
import bz2
import json
import requests as rq
import time

In [18]:
dump = "/datasets_1/sagarj/reddit_all_2017/RC_2017-02.bz2"

In [19]:
openedDump = bz2.BZ2File(dump, "r")

In [8]:
count = 0
for line in openedDump:
    if count > 10:
        break
    print repr(line)
    count+=1

'{"author_flair_text":null,"id":"dd61ge9","edited":false,"subreddit_id":"t5_2zgmb","stickied":false,"retrieved_on":1488453514,"author_flair_css_class":null,"created_utc":1485907200,"controversiality":0,"link_id":"t3_5qxrvb","parent_id":"t1_dd3vflb","subreddit":"AshevilleMusic","author":"Durgroth","distinguished":null,"body":"I have no idea what these places are other than the Mothlight, but I\'ll definitely check them out!","gilded":0,"score":1}\n'
'{"edited":false,"stickied":false,"retrieved_on":1488453514,"subreddit_id":"t5_2qh33","id":"dd61gea","author_flair_text":null,"distinguished":null,"author":"SlothOfDoom","parent_id":"t1_dd5zw81","subreddit":"funny","score":16,"gilded":0,"body":"She was also one of the very first Federation ships built specifically as a warship (although officially it is an \'escort\' so the Federation can still look touchy-feely)","created_utc":1485907200,"author_flair_css_class":null,"link_id":"t3_5r9d4o","controversiality":0}\n'
'{"edited":false,"stickied"

In [9]:
def getJson(url):
    session_headers = {
    'Host': 'www.reddit.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #'Referer': 'http://localhost:9000/notebooks/Reddit/Notebooks/SuicideWatchSubreddit.ipynb'
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
    'Cookie': "loid=00000000000gslzvdi.2.1507591930964.Z0FBQUFBQlozQWI3M243MTdFMUgzNWtQNzNVbzg5Z3NXYmxpLUdRTWVXdEpWQzBfbkxBSVNNVVF3dE1zQlN5cVMzeGtxZXlmT0JQY2JvTld2NWlVeGlGeFpkNnBYM2ZIYlNNR2xMWGFFS3hieDZpWEJ5OVlyTng0MUFZZjNYdjQ0ZURFNTFlSmlVRGw; edgebucket=T8nkTF0z7WPnNzlE8B; __gads=ID=ab9c1b0fcb7eaefc:T=1507591936:S=ALNI_Mao2tvEoyJzEDEuxxhfZBlOQHXlsA; eu_cookie_v2=3; _ga=GA1.2.1088810883.1507591931; __utma=55650728.1088810883.1507591931.1507912304.1507912553.14; __utmz=55650728.1507912553.14.11.utmcsr=reddit|utmccn=(not%20set)|utmcmd=hot|utmcct=comments; aa=1; _recentclicks2=t3_7cfzsx%2Ct3_7cfw08%2Ct3_7cdofu%2Ct3_7caog1%2Ct3_7cdvud; _recent_srs=t5_2qpzs%2Ct5_2qh1q%2Ct5_2qh49%2Ct5_2rtff%2Ct5_36buk%2Ct5_2r7yd%2Ct5_2rks3%2Ct5_3j2jr%2Ct5_2sumw%2Ct5_2xp2o; pc=r3; session_tracker=tPgx9U9yPt99jYRDqC.0.1510585492994.Z0FBQUFBQmFDYlNWcWZHclI5WWxDWW9kQ0dURk9FRU9BZjFJTURWZElzaldRbkROaEJUOWE1czRQUWxPUzUwOFFwY3R3ckI5VkNDUlhJc01NVEtEeHVWd25CdUpXWEI1X012RkJaS3cyNXdNbk5vOE9DSXp5ZlhuTzF3a2pxT2dLZ3laMDhtQlhib3k; initref=localhost"
    }

    client = rq.session()
    resp = client.get(url , headers=session_headers)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print "Failed to get page"
        return None
def getSubredditId(json):
    if type(json) == dict:
        sub_id = json['data']['children'][0]['data']['subreddit_id']
        return sub_id
    else: 
        print "Invalid Format"
        return None

def makeSubUrl(name):
    return "https://www.reddit.com/r/" + name + "/.json"

def getSubIds(RedditList):
    subIds = {}
    for name in RedditList:
        json = getJson(makeSubUrl(name))
        subIds[name] = getSubredditId(json)
    return subIds

def filterDumpbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print "=============Parsing============"
    print subID_dict
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit'] in subID_dict:
            if lineData['subreddit'] not in returnData:
                returnData[lineData['subreddit']] = []
            postId = lineData['parent_id'].strip().split('_')[1]
            
            if postId not in returnData[lineData['subreddit']]:
                returnData[lineData['subreddit']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print "Done parsing %d posts in %d seconds"%(count,(end-st))
            st = time.time()
        count+=1
    return returnData

In [11]:
# subreddits = ["mentalhealth","cripplingalcoholism" , "disorder" , "Health" , "addiction" , "ADHD", "depression",'TheDonald' , 'SuicideWatch'] 
subreddits = ["changemyview","gue" ]

In [12]:
subID_dict = getSubIds(subreddits)

In [13]:
type(subID_dict)

dict

In [14]:
postData = filterDumpbySub(dump,subID_dict)

=============Parsing============
{'changemyview': u't5_2w2s8', 'gue': u't5_2sgcn'}
Done parsing 0 posts in 0 seconds
Done parsing 100000 posts in 2 seconds
Done parsing 200000 posts in 2 seconds
Done parsing 300000 posts in 2 seconds
Done parsing 400000 posts in 2 seconds
Done parsing 500000 posts in 2 seconds
Done parsing 600000 posts in 2 seconds
Done parsing 700000 posts in 2 seconds
Done parsing 800000 posts in 2 seconds
Done parsing 900000 posts in 2 seconds
Done parsing 1000000 posts in 2 seconds
Done parsing 1100000 posts in 2 seconds
Done parsing 1200000 posts in 2 seconds
Done parsing 1300000 posts in 2 seconds
Done parsing 1400000 posts in 2 seconds
Done parsing 1500000 posts in 2 seconds
Done parsing 1600000 posts in 2 seconds
Done parsing 1700000 posts in 2 seconds
Done parsing 1800000 posts in 2 seconds
Done parsing 1900000 posts in 2 seconds
Done parsing 2000000 posts in 2 seconds
Done parsing 2100000 posts in 2 seconds
Done parsing 2200000 posts in 2 seconds
Done parsing

Done parsing 20100000 posts in 2 seconds
Done parsing 20200000 posts in 2 seconds
Done parsing 20300000 posts in 2 seconds
Done parsing 20400000 posts in 2 seconds
Done parsing 20500000 posts in 2 seconds
Done parsing 20600000 posts in 2 seconds
Done parsing 20700000 posts in 2 seconds
Done parsing 20800000 posts in 2 seconds
Done parsing 20900000 posts in 2 seconds
Done parsing 21000000 posts in 2 seconds
Done parsing 21100000 posts in 2 seconds
Done parsing 21200000 posts in 2 seconds
Done parsing 21300000 posts in 2 seconds
Done parsing 21400000 posts in 2 seconds
Done parsing 21500000 posts in 2 seconds
Done parsing 21600000 posts in 2 seconds
Done parsing 21700000 posts in 2 seconds
Done parsing 21800000 posts in 2 seconds
Done parsing 21900000 posts in 2 seconds
Done parsing 22000000 posts in 2 seconds
Done parsing 22100000 posts in 2 seconds
Done parsing 22200000 posts in 2 seconds
Done parsing 22300000 posts in 2 seconds
Done parsing 22400000 posts in 2 seconds
Done parsing 225

Done parsing 40100000 posts in 2 seconds
Done parsing 40200000 posts in 2 seconds
Done parsing 40300000 posts in 2 seconds
Done parsing 40400000 posts in 2 seconds
Done parsing 40500000 posts in 2 seconds
Done parsing 40600000 posts in 2 seconds
Done parsing 40700000 posts in 2 seconds
Done parsing 40800000 posts in 2 seconds
Done parsing 40900000 posts in 2 seconds
Done parsing 41000000 posts in 2 seconds
Done parsing 41100000 posts in 2 seconds
Done parsing 41200000 posts in 2 seconds
Done parsing 41300000 posts in 2 seconds
Done parsing 41400000 posts in 2 seconds
Done parsing 41500000 posts in 2 seconds
Done parsing 41600000 posts in 2 seconds
Done parsing 41700000 posts in 2 seconds
Done parsing 41800000 posts in 2 seconds
Done parsing 41900000 posts in 2 seconds
Done parsing 42000000 posts in 2 seconds
Done parsing 42100000 posts in 2 seconds
Done parsing 42200000 posts in 2 seconds
Done parsing 42300000 posts in 2 seconds
Done parsing 42400000 posts in 2 seconds
Done parsing 425

Done parsing 60100000 posts in 2 seconds
Done parsing 60200000 posts in 2 seconds
Done parsing 60300000 posts in 2 seconds
Done parsing 60400000 posts in 2 seconds
Done parsing 60500000 posts in 2 seconds
Done parsing 60600000 posts in 2 seconds
Done parsing 60700000 posts in 2 seconds
Done parsing 60800000 posts in 2 seconds
Done parsing 60900000 posts in 2 seconds
Done parsing 61000000 posts in 2 seconds
Done parsing 61100000 posts in 2 seconds
Done parsing 61200000 posts in 2 seconds
Done parsing 61300000 posts in 2 seconds
Done parsing 61400000 posts in 2 seconds
Done parsing 61500000 posts in 2 seconds
Done parsing 61600000 posts in 2 seconds
Done parsing 61700000 posts in 2 seconds
Done parsing 61800000 posts in 2 seconds
Done parsing 61900000 posts in 2 seconds
Done parsing 62000000 posts in 2 seconds
Done parsing 62100000 posts in 2 seconds
Done parsing 62200000 posts in 2 seconds
Done parsing 62300000 posts in 2 seconds
Done parsing 62400000 posts in 2 seconds
Done parsing 625

In [20]:
postData['changemyview']

[u'5rbww1',
 u'dd617n3',
 u'dd5yiav',
 u'dd5zhs1',
 u'dd61be3',
 u'dd5swm4',
 u'dd5ynkh',
 u'5rb7bq',
 u'dd5cexa',
 u'dd61bxi',
 u'5rbt16',
 u'dd60t9b',
 u'5rc0p0',
 u'dd61ky2',
 u'dd5y2ux',
 u'dd618e6',
 u'dd5w4dk',
 u'dd5yxbd',
 u'dd5zssz',
 u'dd4m8ex',
 u'dd61oz5',
 u'dd61kh5',
 u'dd60ds1',
 u'dd5r1cb',
 u'dd5rple',
 u'5rbb56',
 u'dd4nz5h',
 u'dd5rbl8',
 u'dd61gh5',
 u'dd60k8j',
 u'dd61fzw',
 u'dd5wsl3',
 u'dd605mv',
 u'dd6209p',
 u'dd60tva',
 u'dd5ocdr',
 u'5radnb',
 u'dd4ve69',
 u'dd614ti',
 u'dd6067p',
 u'dd61q3t',
 u'dd61q6m',
 u'dd61h9u',
 u'5rbf5l',
 u'dd624cq',
 u'dd61tyz',
 u'dd601z9',
 u'dd60c7g',
 u'dd61yv9',
 u'5r9rzx',
 u'dd60sa3',
 u'dd5zx01',
 u'dd5urlp',
 u'dd60m2r',
 u'dd5izyh',
 u'5r9t4p',
 u'5rc58f',
 u'5r9633',
 u'dd6235x',
 u'dd61xm0',
 u'dd5x6bm',
 u'dd62j3s',
 u'dd61au6',
 u'dd627mb',
 u'dd606d9',
 u'dd62nb3',
 u'dd616io',
 u'dd604qf',
 u'dd5ztq3',
 u'dd61hmt',
 u'5rc6a3',
 u'dd5xnv7',
 u'dd61m93',
 u'dd624u0',
 u'dd62b91',
 u'dd04pha',
 u'dd5xgqn',
 u'dd6241p'

In [22]:
# import pickle as pkl
# with open("ChangemyView2017.pkl",'w') as f:
#     pkl.dump(postData['changemyview'],f)

In [55]:
# with open("Filtered_posts_2017.json",'w') as f:
#     json.dump(postData,f)